<a href="https://colab.research.google.com/github/gabrielutomo/Youtube-Music-Hits/blob/main/UTSPRAKTIKDIMASfix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import library yang dibutuhkan
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 1. Memuat Dataset
# Ganti dengan nama file Anda jika berbeda
file_path = '/content/DP.Proyek1.xlsx'

try:
    # Coba baca file Excel
    df = pd.read_excel(file_path)
except Exception as e:
    print(f"Error saat membaca file Excel: {e}")
    print("Pastikan file yang diberikan adalah format Excel (.xlsx atau .xls) yang valid.")
    exit()

# Tampilkan informasi awal data
print("--- 1. Informasi Awal Dataset ---")
print(df.head())
print("\n")
print(df.info())
print("\n" + "="*50 + "\n")

# 2. Preprocessing Data

# Pilih fitur (X) dan target (y).
# Asumsi: Kita akan memprediksi 'Status Penanaman Modal' (Target/y)
# berdasarkan 'Uraian Skala Usaha' dan 'Uraian Risiko Proyek' (Fitur/X)
# Anda bisa menyesuaikan kolom X dan y sesuai kebutuhan.
TARGET_COLUMN = 'Uraian Status Penanaman Modal'
FEATURE_COLUMNS = ['Uraian Skala Usaha', 'Uraian Risiko Proyek']

# Hapus baris dengan nilai yang hilang pada kolom yang dipilih
df_clean = df.dropna(subset=[TARGET_COLUMN] + FEATURE_COLUMNS).copy()
print("Jumlah data setelah membersihkan missing values:", len(df_clean))
print("\n" + "="*50 + "\n")

# Menggunakan LabelEncoder untuk mengkonversi kolom kategorikal menjadi numerik
# Ini diperlukan karena Naive Bayes (terutama GaussianNB) bekerja dengan angka.
le = LabelEncoder()
encoded_features = []

# Konversi kolom Target (y) Mengubah Target Menjadi Angka & Mengubah Label Menjadi Angka
df_clean['target_encoded'] = le.fit_transform(df_clean[TARGET_COLUMN])
target_names = le.classes_ # Get the target names from the target column
print(f"Mapping {TARGET_COLUMN}: {dict(zip(le.classes_, le.transform(le.classes_)))}")


# Konversi kolom Fitur (X) mengubah kolom fitur yang berisi teks menjadi angka
for col in FEATURE_COLUMNS:
    encoded_col_name = f'{col}_encoded'
    # Check if the column exists in df_clean before fitting LabelEncoder
    if col in df_clean.columns:
        le_feature = LabelEncoder()
        df_clean[encoded_col_name] = le_feature.fit_transform(df_clean[col])
        encoded_features.append(encoded_col_name)
        print(f"Mapping {col}: {dict(zip(le_feature.classes_, le_feature.transform(le_feature.classes_)))}")
    else:
        print(f"[PERINGATAN] Kolom '{col}' tidak ditemukan di DataFrame. Lewati encoding.")

print("\nKolom Fitur yang Digunakan Setelah Encoding:", encoded_features)
print("\n" + "="*50 + "\n")


# 3. Pembagian Data Latih dan Data Uji untuk model naivey baiyes
X = df_clean[encoded_features]
y = df_clean['target_encoded']
# X kolom encoded (yang sudah diubah jadi angka)
# Y adalah label yang sudah diubah menjadi angka.

# Split data menjadi 80% data latih (train) dan 20% data uji (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Jumlah data latih (train): {len(X_train)}")
print(f"Jumlah data uji (test): {len(X_test)}")
print("\n" + "="*50 + "\n")


# 4. Pelatihan Model Naive Bayes
# Menggunakan Gaussian Naive Bayes (sesuai untuk fitur numerik yang kontinu/diskret hasil encoding)
model_nb = GaussianNB()

# Latih model menggunakan data latih
model_nb.fit(X_train, y_train)

print("Model Gaussian Naive Bayes telah berhasil dilatih.")
print("\n" + "="*50 + "\n")


# 5. Prediksi dan Evaluasi Model
# Prediksi pada data uji Naive Bayes membuat tebakan/prediksi terhadap data uji.
y_pred = model_nb.predict(X_test)

# Evaluasi Akurasi Membandingkan hasil prediksi (y_pred) dengan jawaban asli (y_test).
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model Naive Bayes: {accuracy:.4f}")

# Laporan Klasifikasi
print("\n--- Laporan Klasifikasi ---")
print(classification_report(y_test, y_pred, target_names=target_names, zero_division=0))

print("\n" + "="*50 + "\n")


# 6. Contoh Prediksi Baru (Opsional)

# Data input baru harus memiliki format yang sama dengan data latih (setelah encoding)
# Contoh: 'Usaha Besar' dan 'Tinggi'
# Kita harus mencari nilai encoding untuk data input ini.
new_data_input = pd.DataFrame({
    'Uraian Skala Usaha': ['Usaha Besar', 'Usaha Menengah', 'Usaha Kecil'],
    'Uraian Risiko Proyek': ['Tinggi', 'Menengah Rendah', 'Rendah'] # Changed 'Menengah' to 'Menengah Rendah'
})

# Lakukan encoding untuk data baru
new_data_encoded = pd.DataFrame()
try:
    # Menggunakan LabelEncoder yang sudah di-fit sebelumnya
    # Make sure to use the correct LabelEncoder instance for each column
    le_usaha = LabelEncoder()
    le_usaha.fit(df_clean['Uraian Skala Usaha'])
    new_data_encoded['Uraian Skala Usaha_encoded'] = le_usaha.transform(new_data_input['Uraian Skala Usaha'])

    le_risiko = LabelEncoder()
    le_risiko.fit(df_clean['Uraian Risiko Proyek'])
    new_data_encoded['Uraian Risiko Proyek_encoded'] = le_risiko.transform(new_data_input['Uraian Risiko Proyek'])


    # Prediksi
    new_predictions_encoded = model_nb.predict(new_data_encoded)

    # Dekode hasil prediksi kembali ke label asli
    predicted_labels = le.inverse_transform(new_predictions_encoded)

    # Tampilkan hasil
    new_data_input['Prediksi Status Penanaman Modal'] = predicted_labels
    print("--- Contoh Prediksi Baru ---")
    print(new_data_input)

except Exception as e:
    print("\n[PERINGATAN] Gagal melakukan contoh prediksi baru. Pastikan semua nilai pada data baru ada di data latih.")
    print(f"Error: {e}")

--- 1. Informasi Awal Dataset ---
   No.                Id Proyek Uraian_Jenis_Proyek           Nib  \
0  1.0  R-201912162228268839132               Utama  8.120312e+12   
1  2.0  R-201912170304070662874               Utama  9.120013e+12   
2  3.0  R-201912170644105256722               Utama  8.120211e+12   
3  4.0  R-201912170653220266679           Pendukung  9.120108e+12   
4  5.0  R-201912170722300003719               Utama  9.120203e+12   

      Nama Perusahaan Tanggal Terbit Oss Uraian Status Penanaman Modal  \
0      APOTEK TEGAR 2         13/12/2018                          PMDN   
1  BERKAH CINTA ABADI         19/11/2019                          PMDN   
2         JOHN MARTIN         18/12/2018                          PMDN   
3           RODA JAYA         15/01/2019                          PMDN   
4      RIO JAYA MOTOR         26/02/2019                          PMDN   

  Uraian Jenis Perusahaan Uraian Risiko Proyek       nama_proyek  ...   Kbli  \
0              Perorangan 